## Neural Collaborative Filtering

### Importing Libraries

In [1]:
import sys
import os
import shutil

# Pandas and Numpy is used for efficient handling of arrays.
import pandas as pd
import numpy as np


from recommenders.utils.timer import Timer
from recommenders.datasets.python_splitters import python_chrono_split

# importing the dataset
from recommenders.datasets import movielens
from recommenders.models.ncf.dataset import Dataset as NCFDataset

# Importing the NCF model class from the recommenders library
from recommenders.models.ncf.ncf_singlenode import NCF

# importing the evaluation metrics
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items)
from recommenders.utils.constants import SEED as DEFAULT_SEED


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
Pandas version: 1.4.4


### Loading the Dataset

We will be using the movielens dataset. It contains the user, movie and the rating given by the user.

In [2]:
#TODO Load the CSV file into a dataframe
df = pd.read_csv('cf_final2.csv')
df= df.rename(columns={"user":"userID", 'item':"itemID", "label":"rating", "song_by": "song"})
df=df[["userID","itemID","rating","timestamp","song"]]
df.head(5)

,userID,itemID,rating,timestamp,song
0,577,30377,0.172815,1.447978e+09,The Safety Dance by Men Without Hats
1,147,15910,0.127811,1.441325e+09,Endless Summer by Grizfolk
2,690,40737,0.097224,1.413331e+09,Castaway by Zac Brown Band
3,59,40737,0.103762,1.404950e+09,Castaway by Zac Brown Band
4,415,49732,0.102779,1.402963e+09,Islands In the Stream by Dolly Parton


In [7]:
# top k items to recommend
TOP_K = 10

# Model parameters
# Number of iterations during the training process
EPOCHS = 25
# Batch size means how many user-item pairs you want to predict at once
BATCH_SIZE = 256

# Setting seed to remove any stochasticity and reproduce results
SEED = DEFAULT_SEED  # Set N

In [3]:
# Splitting the dataset.
# 75% will be used during training and 25% will be used during testing

train, test = python_chrono_split(df, 0.75)


In [4]:
# Filtering out users and items in the test set that do not appear in the training set.
# This is done so that we can see if our model has learnt user's previous item interactions and can recommend relevant items.

test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

# Creating a test set which only contains the last interaction for each user. Remaining data of the user is used in the train set
leave_one_out_test = test.groupby("userID").last().reset_index()


In [5]:
# Writing the data into csv files

train_file = "./train.csv"
test_file = "./test.csv"
leave_one_out_test_file = "./leave_one_out_test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

In [8]:
data = NCFDataset(train_file=train_file, test_file=leave_one_out_test_file, seed=SEED, overwrite_test_file_full=True)

INFO:recommenders.models.ncf.dataset:Indexing ./train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./leave_one_out_test.csv ...
INFO:recommenders.models.ncf.dataset:Creating full leave-one-out test file ./leave_one_out_test_full.csv ...
100%|██████████| 717/717 [00:06<00:00, 104.28it/s]
INFO:recommenders.models.ncf.dataset:Indexing ./leave_one_out_test_full.csv ...


### Training the NCF Model

In [14]:
model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

C:\Users\darvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [15]:
# Fitting the model on the training data. This can take 4 to 17 mins. Depending on n_epochs and if you are running on CPU/GPU.

with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [11.02s]: train_loss = 0.173687 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [11.10s]: train_loss = 0.096262 


Took 279.8980634 seconds for training.


### Prediction and Evaluation

Getting predictions from our trained model. We are converting it to a pandas dataframe later.

In [17]:
predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]


predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

,userID,itemID,prediction
0,0,26234,0.000007
1,1,50800,0.997452
2,1,16571,0.028526
3,1,17377,0.859025
4,2,8182,0.196734


In this step we are removing items that have already been rated by the user. We do not want to recommend the same item again to the user.

In [18]:
with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 83.99911450000002 seconds for prediction.


#### MAP

It is the average precision for each user normalized over all users.

In [19]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"MAP: {eval_map}")

MAP: 0.000952636391966936


#### NDCG

Normalized Discounted Cumulative Gain (NDCG) - evaluates how well the predicted items for a user are ranked based on relevance


In [20]:
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"NDCG: {eval_ndcg}")

NDCG: 0.0037502676510127297


#### Precision Recall

Precision - this measures the proportion of recommended items that are relevant

Recall - this measures the proportion of relevant items that are recommended

In [21]:
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
print(f"Precision: {eval_precision} \n Recall: {eval_recall}")

Precision: 0.003626220362622037 
 Recall: 0.0034648038309126185
